In [ ]:
import holoviews as hv
import numpy as np
import pandas as pd
from holoviews import opts

from cross import cross_energy_matrix
from curvature import curvature_energy_matrix, segment_adjacent_pairs
from generator import SimpleEventGenerator
from reconstruct import annealing_curve, update_layer_grad, energy, energy_gradient, should_stop
from plot import make_tracks_3d
from segment import gen_segments_all

In [ ]:
import pickle

with open('workdir/514452.pkl', 'rb') as fh:
    result = pickle.load(fh)

best_run = result.get_runs_by_id(result.get_incumbent_id())[-1]
best_run

In [ ]:
config = result.get_id2config_mapping()[best_run.config_id]['config']
THRESHOLD = config['threshold']
config

In [ ]:
N_TRACKS = 20
N_EVENTS = 1

eventgen = SimpleEventGenerator(
    seed=13, field_strength=0.8, noisiness=10, box_size=.5
).gen_many_events(N_EVENTS, N_TRACKS)

hits, track_segments = next(eventgen)
hits

In [ ]:
pos = hits[['x', 'y', 'z']].values

In [ ]:
seg = gen_segments_all(hits)
len(track_segments), len(seg)

In [ ]:
perfect_act = np.zeros(len(seg))
track_segment_set = set(tuple(s) for s in track_segments)
is_in_track = np.array([tuple(s) in track_segment_set for s in seg])
perfect_act[is_in_track] = 1

In [ ]:
pairs = segment_adjacent_pairs(seg)

crossing_matrix = cross_energy_matrix(seg, pos, config['cosine_min_allowed'])

curvature_matrix = curvature_energy_matrix(pos, seg, pairs,
                                           config['cosine_power'], config['cosine_min_rewarded'],
                                           config['distance_power'])

e_matrix = config['alpha'] / 2 * crossing_matrix - config['gamma'] / 2 * curvature_matrix

In [ ]:
temp_curve = annealing_curve(config['tmin'], config['tmax'],
                             config['anneal_steps'], config['total_steps'] - config['anneal_steps'])

tcdf = pd.DataFrame({'temp': temp_curve})
tcdf.index.name = 'step'
tcdf.plot()

In [ ]:
act = np.full(len(seg), config['starting_act'])
acts = [act.copy()]
for i, t in enumerate(temp_curve):
    grad = energy_gradient(e_matrix, act)
    update_layer_grad(act, grad, t, config['dropout'], config['learning_rate'], config['bias'])
    acts.append(act.copy())

In [ ]:
energy_history = []
for act in acts:
    ef = config['alpha'] / 2 * energy(crossing_matrix, act)
    ec = config['gamma'] / 2 * energy(curvature_matrix, act) # inverted to positive
    energy_history.append([ec, ef])
energy_history = pd.DataFrame(energy_history, columns=['E_curve', 'E_fork'])
energy_history.index.name = 'step'
energy_history.plot(logy=True)
pass

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
small_history = pd.DataFrame([
    (
        precision_score(perfect_act, act>THRESHOLD, zero_division=0),
        recall_score(perfect_act, act>THRESHOLD, zero_division=0),
        f1_score(perfect_act, act>THRESHOLD, zero_division=0),
        roc_auc_score(perfect_act, act),
        average_precision_score(perfect_act, act),
        act.mean(),
        ((act - perfect_act) ** 2).mean()
    ) for act in acts],
    columns=['precision', 'recall', 'f1', 'roc_auc', 'ap', 'mean_act', 'dist_perfect'])
small_history.index.name = 'step'
small_history.plot()

In [ ]:
from sklearn.metrics import precision_recall_curve, PrecisionRecallDisplay, average_precision_score
average_precision = average_precision_score(perfect_act, act)
precision, recall, _ = precision_recall_curve(perfect_act, act)
PrecisionRecallDisplay(precision=precision, recall=recall, average_precision=average_precision).plot()

In [ ]:
from sklearn.metrics import roc_curve, RocCurveDisplay
fpr, tpr, _ = roc_curve(perfect_act, act)
RocCurveDisplay(fpr=fpr, tpr=tpr).plot()

In [ ]:
from sklearn.metrics import det_curve, DetCurveDisplay
fpr, fnr, _ = det_curve(perfect_act, act)
DetCurveDisplay(fpr=fpr, fnr=fnr).plot()

In [ ]:
n_steps = min(16, len(acts))
steps = np.linspace(0, len(acts) - 1, n_steps, dtype=int)

tracks_3d = []
tracks_projection = []
tracks_by_track = []
for i in steps:
    tp, fp, tn, fn = make_tracks_3d(pos, seg, acts[i], perfect_act, THRESHOLD)
    vdims = ['act', 'perfect_act', 'positive', 'true']
    xyz = hv.Overlay([
        hv.Scatter3D(hits[hits.track == -1], kdims=['x', 'y', 'z'], label='noise', group='hits'),
        hv.Scatter3D(hits[hits.track != -1], kdims=['x', 'y', 'z'], label='hits', group='hits'),
        hv.Path3D(tp, vdims=vdims, label='tp', group='tracks'),
        hv.Path3D(fp, vdims=vdims, label='fp', group='tracks'),
        hv.Path3D(fn, vdims=vdims, label='fn', group='tracks')
    ])
    tracks_3d.append(xyz)

    projection = hv.Points(hits[hits.track != -1], kdims=['x', 'y'], label='hits', group='hits') * \
                 hv.Points(hits[hits.track == -1], kdims=['x', 'y'], label='noise', group='hits') * \
                 hv.Path(tp, kdims=['x', 'y'], vdims=vdims, label='tp', group='tracks') * \
                 hv.Path(fp, kdims=['x', 'y'], vdims=vdims, label='fp', group='tracks') * \
                 hv.Path(fn, kdims=['x', 'y'], vdims=vdims, label='fn', group='tracks')
    tracks_projection.append(projection)

    nodes = hv.Nodes(hits, kdims=['track', 'layer', 'index'])
    no_tn = np.logical_or(acts[i] > THRESHOLD, perfect_act > THRESHOLD)
    graph = hv.Graph(((*seg[no_tn].transpose(), acts[i][no_tn], perfect_act[no_tn]), nodes),
                     vdims=['act', 'perfect_act'])
    tracks_by_track.append(hv.Overlay([graph]))


In [ ]:
hv.extension('matplotlib')
tracks_3d[-1].opts(
    opts.Scatter3D(c='black'),
    opts.Scatter3D('Hits.Noise', c='red'),
    opts.Path3D(color='black', show_legend=True),
    opts.Path3D('Tracks.fp', color='red'),
    opts.Path3D('Tracks.fn', color='cyan'),
    opts.Overlay(legend_position='right', fig_size=400),
)

In [ ]:
hv.extension('bokeh')
track_history = \
    hv.HoloMap(
        {s: hv.Histogram(np.histogram(acts[s][perfect_act > THRESHOLD], bins=32, range=(0, 1))) for s in
         np.linspace(0, len(acts) - 1, min(64, len(acts)), dtype=int)},
        kdims='step', group="Activation", label="On track") + \
    hv.HoloMap(
        {s: hv.Histogram(np.histogram(acts[s][perfect_act < THRESHOLD], bins=32, range=(0, 1))) for s in
         np.linspace(0, len(acts) - 1, min(64, len(acts)), dtype=int)},
        kdims='step', group="Activation", label="Off track")

track_history.opts(
    opts.Histogram(width=400, height=400)
)

In [ ]:
hv.extension('plotly')
tracks_3d[-1].opts(
    opts.Scatter3D(size=2, color='black'),
    opts.Scatter3D('Hits.Noise', color='red'),
    opts.Path3D(color='black'),
    opts.Path3D('Tracks.fp', color='red'),
    opts.Path3D('Tracks.fn', color='cyan'),
    opts.Overlay(width=600, height=600)
)

In [ ]:
hv.extension('matplotlib')
track_history = hv.NdLayout(
    {s: tracks_3d[i] for i, s in enumerate(steps)},
    kdims='step'
)
track_history.opts(
    opts.Scatter3D(c='black'),
    opts.Scatter3D('Hits.Noise', c='red'),
    opts.Path3D(color='black', show_legend=True),
    opts.Path3D('Tracks.fp', color='red'),
    opts.Path3D('Tracks.fn', color='cyan'),
    opts.Overlay(legend_position='right'),
    opts.NdLayout(fig_size=200)
).cols(4)

In [ ]:
hv.extension('plotly')
track_history = hv.HoloMap(
    {s: tracks_3d[i] for i, s in enumerate(steps)},
    kdims='step'
)
track_history.opts(
    opts.Scatter3D(size=2, color='black'),
    opts.Scatter3D('Hits.Noise', color='red'),
    opts.Path3D(color='black'),
    opts.Path3D('Tracks.fp', color='red'),
    opts.Path3D('Tracks.fn', color='cyan'),
    opts.Overlay(width=700, height=700)
)
# hv.save(track_history, 'track_history_3d2.html', fmt='scrubber')

In [ ]:
# hv.extension('matplotlib')
# track_history = hv.HoloMap(
#     {s: tracks_projection[i] for i, s in enumerate(steps)},
#     kdims='step'
# )
# track_history.opts(
#     opts.Points(color='black'),
#     opts.Points('Hits.Noise', color='black'),
#     opts.Path(color='black', show_legend=True),
#     opts.Path('Tracks.fp', color='red'),
#     opts.Path('Tracks.fn', color='cyan'),
#     opts.Overlay(legend_position='right', fig_size=256),
# )

In [ ]:
hv.extension('bokeh')
track_history = hv.HoloMap(
    {s: tracks_projection[i] for i, s in enumerate(steps)},
    kdims='step'
)
track_history.opts(
    opts.Points(color='black', size=4),
    opts.Points('Hits.Noise', color='red'),
    opts.Path(color='black', show_legend=True),
    opts.Path('Tracks.fp', color='red'),
    opts.Path('Tracks.fn', color='cyan'),
    opts.Overlay(legend_position='right', width=700, height=700)
)
